<a href="https://colab.research.google.com/github/aithaprasad/related-languages/blob/main/Uday_NLP_5_anything_goes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_source = open('train-source.txt', 'r')

In [ ]:
train_source_sentences = []
temp = []
for line in train_source:
  token = line.rstrip("\n")
  if token == '<s>': temp = []
  elif token == '</s>': train_source_sentences.append(temp)
  else: temp.append(line.split('\n')[0])

In [ ]:
train_target = open('train-target.txt', 'r')

In [ ]:
train_target_sentences = []
temp = []
for line in train_target:
  token = line.rstrip("\n")
  if token == '<s>': temp = []
  elif token == '</s>': train_target_sentences.append(temp)
  else: temp.append(line.split('\n')[0])
temp = []

In [ ]:
len(train_source_sentences) == len(train_target_sentences) 

True

In [ ]:
from keras.preprocessing.text import Tokenizer
def tokenization(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [ ]:
source_tokenizer = tokenization(train_source_sentences)
source_vocab_size = len(source_tokenizer.word_index) + 1

In [ ]:
mini, maxi = 0, 0
for sent in train_source_sentences:
  sizes = len(sent)
  if len(sent) <= 25: mini += 1
  else: maxi += 1
print(mini)
print(maxi)

35761
9410


In [ ]:
source_max_length = 25
print('Source Vocabulary Size: %d' % source_vocab_size)

Source Vocabulary Size: 31038


In [ ]:
target_tokenizer = tokenization(train_target_sentences)
target_vocab_size = len(target_tokenizer.word_index) + 1
target_max_length = 25
print('Target Vocabulary Size: %d' % target_vocab_size)

Target Vocabulary Size: 25738


In [ ]:
from keras_preprocessing.sequence import pad_sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [ ]:
from sklearn.model_selection import train_test_split

# split data into train and test set
source_train, source_test = train_test_split(train_source_sentences, test_size=0.01, random_state = 42)
target_train, target_test = train_test_split(train_target_sentences, test_size=0.01, random_state = 42)

In [ ]:
# prepare training data
trainX = encode_sequences(source_tokenizer, source_max_length, source_train)
trainY = encode_sequences(target_tokenizer, target_max_length, target_train)

In [ ]:
# prepare validation data
testX = encode_sequences(source_tokenizer, source_max_length, source_test)
testY = encode_sequences(target_tokenizer, target_max_length, target_test)

https://www.analyticsvidhya.com/blog/2019/01/neural-machine-translation-keras/

Utilized the model from here.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
# build NMT model
def define_model(in_vocab,out_vocab, in_timesteps,out_timesteps,units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dense(out_vocab, activation='softmax'))
    return model

In [ ]:
model = define_model(source_vocab_size, target_vocab_size, source_max_length, target_max_length, 128)

In [ ]:
from keras import optimizers
rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=1, batch_size=16, validation_split = 0.2, 
                    verbose=1)

559/559 [==============================] - 609s 1s/step - loss: 4.7677 - val_loss: 4.2358


In [ ]:
model.save('lstm_bs_16_lr_0.001.h5')

In [ ]:
#model.save('lstm_bs_512_lr_0.1.h5')

In [ ]:
from tensorflow import keras
#loaded_model = keras.models.load_model('lstm_bs_512_lr_0.1.h5')
loaded_model = keras.models.load_model('lstm_bs_64_lr_0.001.h5')

In [ ]:
preds = model.predict(testX.reshape((testX.shape[0],testX.shape[1])))

15/15 [==============================] - 2s 10ms/step


In [ ]:
#predict_x = model.predict(X_test) 
preds=np.argmax(preds,axis=1)

In [ ]:
#single_pred = loaded_model.predict(testX.reshape((testX.shape[0],testX.shape[1])))

In [ ]:
rev_map = dict()
for word, index in target_tokenizer.word_index.items():
  rev_map[index] = word
def get_own_word(n):
  if n not in rev_map: return None
  return rev_map[index]

In [ ]:
len(preds[0])

25738

In [ ]:
preds_text = []
for i in preds:
       temp = []
       for j in range(len(i)):
            t = get_own_word(i[j])#, target_tokenizer)
            if j > 0:
                if (t == get_own_word(i[j-1])) or (t == None): #, target_tokenizer)
                     temp.append('')
                else:
                     temp.append(t)
            else:
                   if(t == None):
                          temp.append('')
                   else:
                          temp.append(t) 

       preds_text.append(' '.join(temp))

In [ ]:
pred_df = pd.DataFrame({'actual' : target_test, 'predicted' : preds_text})

In [ ]:
pred_df.sample(5)

,actual,predicted
388,"[Rinne, Sean, Ó, Fríl, gáire, .]",shéamaisín shéamaisín shéamaisín sh...
325,"[Ach, ba, é, an, rud, a, rinne, mé, a, dhul, a...",shéamaisín shéamaisín shéamaisín ...
394,"[Dúirt, sé, go, brónach, agus, go, dúlaí, ,, ""...",shéamaisín shéamaisín shéamaisín ...
158,"[Shuigh, sé, siar, sa, choirnéal, sin, i, ndia...",shéamaisín shéamaisín shéamaisín ...
155,"[Baineann, an, bhuíóg, fúithi, amuigh, ar, an,...",shéamaisín shéamaisín shéama...
